# Convert to a Ring Optimized skymap

## Overview

In this notebook we will:
- walk through the convertion of an lsst skymap file into a "Ring Optimized" skymap file
- a Ring Optimized skymap (explain)

In [1]:
from pathlib import Path
from skymap_convert import load_pickle_skymap, write_polygons_ra_dec

## Read in raw skymap

In [ ]:
package_root = Path.home() / "skymap-convert"

raw_skymaps_dir = package_root / "tests" / "data" / "raw_skymaps"
raw_skymap_path = raw_skymaps_dir / "skyMap_lsst_cells_v1_skymaps.pickle"

In [3]:
lsst_skymap = load_pickle_skymap(raw_skymap_path)
lsst_skymap

## Write Ring Optimized file

See main "Convert skymaps" notebook for a description of ring skymaps, as well as inner and outer polygons.

The process here is based on a set of assumptions that have been explored in Appendix 1 (Ring skymap assumptions).

These are:
1. All tracts have exactly 4 vertices
1. Except the poles, all tracts are rectangular (to the nearest arcsecond)
1. Within a ring, all tracts share the same upper and lower declination
1. Within a ring, all tracts have the same width, or interval between their RA boundaries
1. Each ring begins calculating their tracts at the same RA

In [4]:
converted_skymaps_dir = package_root / "converted_skymaps" / "ring_optimized"

inner_poly_path = converted_skymaps_dir / "inner_polygons.yaml"
outer_poly_path = converted_skymaps_dir / "outer_polygons.yaml"

In [5]:
from skymap_convert import write_ring_optimized_skymap

write_ring_optimized_skymap(
    lsst_skymap, 
    inner_poly_path, 
    inner=True,
    patches=False,
    skymap_name="lsst_v1"
)

✅ Ring-optimized skymap written to /sdf/home/o/olynn/skymap-to-poly-coords/converted_skymaps/ring_optimized/inner_polygons.yaml


In [6]:
# Use bash to see how big the file is:
!ls -lh {inner_poly_path}

-rw-r--r-- 1 olynn gu 13K Jun 12 23:17 /sdf/home/o/olynn/skymap-to-poly-coords/converted_skymaps/ring_optimized/inner_polygons.yaml


## Verify file

In [7]:
# read the file back in
from skymap_convert import RingOptimizedSkymapReader

ro_skymap = RingOptimizedSkymapReader(inner_poly_path)
print(ro_skymap)

RingOptimizedSkymapReader(skymap_name=lsst_v1, rings=120, poles=2, inner_polygons=True)


In [8]:
# from skymap_convert import IterateTractAndRing

# iter_tr = IterateTractAndRing(ro_skymap.ring_nums)

# for tract, ring in iter_tr:
#     print(f"Tract: {tract}, Ring: {ring}")
#     quad = ro_skymap.get_tract(tract)["quad"]
#     for point in quad:
#         print(f"  Point: {point}")

#     if tract >= 5:
#         break

In [ ]:
from skymap_convert import box_to_convex_polygon, unit_vector3d_to_radec
from lsst.sphgeom import Box
import numpy as np

for tract in lsst_skymap:
    tract_id = tract.getId()

    # Get quad from Ring Optimized skymap.
    ro_quad = ro_skymap.get_tract(tract_id)["quad"]

    # Get poly from original skymap.
    poly = tract.inner_sky_region
    if isinstance(poly, Box):
        poly = box_to_convex_polygon(poly)
    ra_dec_vertices = [[radec[0], radec[1]] for radec in map(unit_vector3d_to_radec, poly.getVertices())]

    for i in range(len(ro_quad)):
        ra_ro, dec_ro = ro_quad[i]
        ra_orig, dec_orig = ra_dec_vertices[i]
        are_close = np.allclose([ra_ro, dec_ro], [ra_orig, dec_orig], rtol=1e-5, atol=1e-5)
        if not are_close:
            print(f"Tract ID: {tract_id}")
            print(f"  Mismatch at vertex {i}: RO ({ra_ro}, {dec_ro}), Orig ({ra_orig}, {dec_orig})")

print(f"Checked {len(lsst_skymap)} tracts for vertex matches.")
print("All vertices match between the Ring Optimized skymap and the original skymap.")